In [2]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)  # voir toutes les colonnes
pd.set_option("display.width", 200)        # affichage plus large
file_path = r"./data/complete_laptop_data.csv"  # à adapter

#Chargement du fichier CSV
df_raw = pd.read_csv(
    file_path,
    sep=",",           # ou ";" si ton CSV est séparé par des points-virgules
    encoding="latin-1",  # ou "latin-1" si tu as des soucis d’accents
    low_memory=False
)
print(df_raw.shape)

#Affichage des 5 premières lignes
df_raw.head()
df_raw.info()
df_raw.columns

(984, 98)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 98 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Unnamed: 0                         984 non-null    int64  
 1   link                               984 non-null    object 
 2   name                               984 non-null    object 
 3   user rating                        690 non-null    float64
 4   Price                              984 non-null    object 
 5   Sales Package                      984 non-null    object 
 6   Model Number                       984 non-null    object 
 7   Part Number                        984 non-null    object 
 8   Model Name                         709 non-null    object 
 9   Series                             787 non-null    object 
 10  Color                              984 non-null    object 
 11  Type                               984 non-null 

Index(['Unnamed: 0', 'link', 'name', 'user rating', 'Price', 'Sales Package', 'Model Number', 'Part Number', 'Model Name', 'Series', 'Color', 'Type', 'Suitable For', 'Power Supply', 'Battery Cell',
       'MS Office Provided', 'Dedicated Graphic Memory Type', 'Dedicated Graphic Memory Capacity', 'Processor Brand', 'Processor Name', 'Processor Generation', 'SSD', 'SSD Capacity', 'RAM',
       'RAM Type', 'Processor Variant', 'Clock Speed', 'Expandable Memory', 'Cache', 'Graphic Processor', 'Number of Cores', 'OS Architecture', 'Operating System', 'Supported Operating System',
       'Mic In', 'USB Port', 'HDMI Port', 'Touchscreen', 'Screen Size', 'Screen Resolution', 'Screen Type', 'Speakers', 'Internal Mic', 'Sound Properties', 'Wireless LAN', 'Bluetooth', 'Dimensions',
       'Weight', 'Disk Drive', 'Finger Print Sensor', 'Keyboard', 'Backlit Keyboard', 'Additional Features', 'Warranty Summary', 'Warranty Service Type', 'Covered in Warranty',
       'Not Covered in Warranty', 'Domesti

Le dataset provenant d’une plateforme e-commerce contient des caractères spéciaux non compatibles avec l’encodage UTF-8 standard.
Nous avons donc utilisé un encodage latin1 / cp1252 afin de garantir un chargement complet et fiable des données. 


# PHASE 2 — Sélection et préparation des colonnes pertinentes (Feature Selection)

Cette phase consiste à réduire le dataset initial en conservant uniquement les colonnes ayant un impact direct sur la recommandation d’un ordinateur portable. Les attributs retenus couvrent l’identité du PC, ses performances matérielles (CPU, GPU, RAM, stockage), le prix, l’usage prévu, l’écran et les aspects liés à la mobilité et à l’énergie. Les colonnes purement descriptives, marketing ou liées au service après-vente ont volontairement été exclues afin de limiter le bruit informationnel et de faciliter la construction du moteur de recommandation.

In [3]:
cols_keep = [
    # Identité
    "link", "name", "Model Name", "Series", "Color", "Type",

    # Prix & avis
    "user rating", "Price",

    # Usage
    "Suitable For",

    # CPU
    "Processor Brand", "Processor Name", "Processor Generation",
    "Processor Variant", "Clock Speed", "Number of Cores", "Cache",

    # RAM
    "RAM", "RAM Type", "RAM Frequency", "Memory Slots",

    # Stockage
    "SSD", "SSD Capacity", "HDD Capacity",

    # GPU
    "Dedicated Graphic Memory Type",
    "Dedicated Graphic Memory Capacity",
    "Graphic Processor", "TGP",

    # Écran
    "Screen Size", "Screen Resolution", "Screen Type",
    "Refresh Rate", "Brightness",

    # Mobilité / énergie
    "Weight", "Dimensions",
    "Battery Backup", "Battery Cell", "Power Supply",

    # OS (simplifié)
    "Operating System",
    #
    "MS Office Provided",
    #Keyboard allumé
    "Backlit Keyboard",
    # Camera
    "Web Camera"
]

missing_cols = [c for c in cols_keep if c not in df_raw.columns]
missing_cols

[]

In [4]:
#Créer le DataFrame métier avec seulement ces colonnes
df_core = df_raw[cols_keep].copy()
df_core.head()
df_core.info()
df_core.shape
df_core.columns


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 41 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   link                               984 non-null    object 
 1   name                               984 non-null    object 
 2   Model Name                         709 non-null    object 
 3   Series                             787 non-null    object 
 4   Color                              984 non-null    object 
 5   Type                               984 non-null    object 
 6   user rating                        690 non-null    float64
 7   Price                              984 non-null    object 
 8   Suitable For                       984 non-null    object 
 9   Processor Brand                    984 non-null    object 
 10  Processor Name                     984 non-null    object 
 11  Processor Generation               690 non-null    object 

Index(['link', 'name', 'Model Name', 'Series', 'Color', 'Type', 'user rating', 'Price', 'Suitable For', 'Processor Brand', 'Processor Name', 'Processor Generation', 'Processor Variant',
       'Clock Speed', 'Number of Cores', 'Cache', 'RAM', 'RAM Type', 'RAM Frequency', 'Memory Slots', 'SSD', 'SSD Capacity', 'HDD Capacity', 'Dedicated Graphic Memory Type',
       'Dedicated Graphic Memory Capacity', 'Graphic Processor', 'TGP', 'Screen Size', 'Screen Resolution', 'Screen Type', 'Refresh Rate', 'Brightness', 'Weight', 'Dimensions', 'Battery Backup',
       'Battery Cell', 'Power Supply', 'Operating System', 'MS Office Provided', 'Backlit Keyboard', 'Web Camera'],
      dtype='object')

In [5]:
output_path = r"C:\Users\Monsieur\OneDrive\Desktop\UNIVERSITE S7\NoSql & BIG DATA\LapMatch\data\laptops_core.csv"

df_core.to_csv(output_path, index=False, encoding="utf-8")

« Nous avons conservé uniquement les attributs influençant directement la performance, l’usage, le prix et l’expérience utilisateur, afin de réduire le bruit informationnel et faciliter la construction du moteur de recommandation. »

# Phase 3 – Nettoyage et normalisation des données

Lors de cette phase, les données sélectionnées ont été nettoyées et normalisées afin d’obtenir un format cohérent et exploitable. Les valeurs textuelles ont été transformées en formats numériques lorsque nécessaire (prix, RAM, capacité de stockage, poids), les unités ont été unifiées, et les formats hétérogènes ont été standardisés. Cette étape garantit la fiabilité des comparaisons entre les ordinateurs et prépare les données pour les calculs de scoring et de filtrage.

In [6]:
import numpy as np
import re   # pour les regex

# Nettoyer Price → numérique
# Avant : " ?2,34,990 "
# Après : 234990 (entier)

# On enlève tout ce qui n'est pas un chiffre, puis on convertit en nombre
df_core["Price"] = (
    df_core["Price"]
    .astype(str)
    .str.replace(r"\D", "", regex=True)  # garde uniquement les chiffres
)

df_core["Price"] = pd.to_numeric(df_core["Price"], errors="coerce")

df_core["Price"].head()

0    234990
1    229990
2    104091
3     87717
4     75990
Name: Price, dtype: int64

In [7]:
#  Nettoyer RAM → RAM_GB
# Avant : "32 GB"
# Après : RAM_GB = 32
df_core["RAM_GB"] = (
    df_core["RAM"]
    .astype(str)
    .str.extract(r"(\d+)", expand=False)  # récupère la partie numérique
)

df_core["RAM_GB"] = pd.to_numeric(df_core["RAM_GB"], errors="coerce")

In [8]:
# Stockage → Storage_Type + Storage_GB

def capacity_to_gb(s):
    if pd.isna(s):
        return np.nan
    s = str(s).strip()
    m = re.search(r"([\d\.]+)\s*(GB|TB)", s, flags=re.I)
    if not m:
        return np.nan
    value = float(m.group(1))
    unit = m.group(2).upper()
    if unit == "TB":
        return value * 1024
    return value

# Calculer SSD_GB et HDD_GB
df_core["SSD_GB"] = df_core["SSD Capacity"].apply(capacity_to_gb)
df_core["HDD_GB"] = df_core["HDD Capacity"].apply(capacity_to_gb)

# Storage_GB = total stockage
df_core["Storage_GB"] = df_core[["SSD_GB", "HDD_GB"]].sum(axis=1, min_count=1)

# Storage_Type
def infer_storage_type(row):
    has_ssd = row["SSD_GB"] > 0 if not pd.isna(row["SSD_GB"]) else False
    has_hdd = row["HDD_GB"] > 0 if not pd.isna(row["HDD_GB"]) else False

    if has_ssd and has_hdd:
        return "SSD+HDD"
    elif has_ssd:
        return "SSD"
    elif has_hdd:
        return "HDD"
    else:
        return np.nan

df_core["Storage_Type"] = df_core.apply(infer_storage_type, axis=1)

# on peut supprimer les colonnes intermédiaires
df_core.drop(columns=["SSD_GB", "HDD_GB"], inplace=True)

In [9]:
# Poids → Weight_KG
# Avant : "2.90 kg"
# Après : Weight_KG = 2.9

df_core["Weight_KG"] = (
    df_core["Weight"]
    .astype(str)
    .str.replace(r"[^0-9\.,]", "", regex=True)  # enlève 'kg', ' KG', etc.
    .str.replace(",", ".", regex=False)        # au cas où virgule décimale
)

df_core["Weight_KG"] = pd.to_numeric(df_core["Weight_KG"], errors="coerce")


In [10]:
# Vérification rapide 
df_core[[
    "Price",
    "RAM", "RAM_GB",
    "SSD", "SSD Capacity", "HDD Capacity",
    "Storage_Type", "Storage_GB",
    "Weight", "Weight_KG"
]].head()

,Price,RAM,RAM_GB,SSD,SSD Capacity,HDD Capacity,Storage_Type,Storage_GB,Weight,Weight_KG
0,234990,32 GB,32,Yes,1 TB,NaN,SSD,1024.0,2.90 kg,2.90
1,229990,32 GB,32,Yes,1 TB,NaN,SSD,1024.0,2.30 kg,2.30
2,104091,16 GB,16,Yes,512 GB,NaN,SSD,512.0,2.48 kg,2.48
3,87717,16 GB,16,Yes,512 GB,NaN,SSD,512.0,2.25 kg,2.25
4,75990,16 GB,16,Yes,512 GB,NaN,SSD,512.0,NaN,NaN


Phase 4 – Enrichissement métier des données

Le dataset a ensuite été enrichi par la création de nouvelles colonnes métier adaptées à la problématique de recommandation. Ces attributs dérivés permettent de caractériser chaque PC selon son usage principal, son niveau de performance et son impact énergétique estimé. Cette étape ajoute une réelle valeur au dataset en traduisant des caractéristiques techniques complexes en informations compréhensibles et exploitables par le moteur de recommandation et, à terme, par le modèle orienté graphe.

In [11]:
# Supprimer les anciennes colonnes texte (RAM / stockage / poids)
cols_drop_initial = [
    "RAM",          # remplacé par RAM_GB
    "SSD",          # intégré dans Storage_Type / Storage_GB
    "SSD Capacity", # intégré dans Storage_GB
    "HDD Capacity", # intégré dans Storage_GB
    "Weight"        # remplacé par Weight_KG
]

df_core.drop(columns=cols_drop_initial, inplace=True, errors="ignore")

In [12]:
import numpy as np
import re
# Screen Size → Screen_Size_Inch (+ optionnel Screen_Size_cm)
def extract_inches(s):
    if pd.isna(s):
        return np.nan
    s = str(s)
    # Cherche "17.3 inch", "15.6 Inch", etc.
    m = re.search(r"([\d\.]+)\s*inch", s, flags=re.I)
    if m:
        return float(m.group(1).replace(",", "."))
    # Si jamais pas "inch", on tente "cm"
    m2 = re.search(r"([\d\.]+)\s*cm", s, flags=re.I)
    if m2:
        cm = float(m2.group(1).replace(",", "."))
        return cm / 2.54
    return np.nan

df_core["Screen_Size_Inch"] = df_core["Screen Size"].apply(extract_inches)
df_core["Screen_Size_cm"] = df_core["Screen_Size_Inch"] * 2.54

In [13]:
#Screen Resolution → Res_Width, Res_Height (+ optionnel Pixels_Total, PPI)
def extract_resolution(s):
    if pd.isna(s):
        return pd.Series([np.nan, np.nan])
    s = str(s)
    # Exemples captés : "1920 x 1080", "1080 x 1920", "1920 x 1080$$Pixels pixel"
    m = re.search(r"(\d+)\s*[xX]\s*(\d+)", s)
    if not m:
        return pd.Series([np.nan, np.nan])
    w = int(m.group(1))
    h = int(m.group(2))
    # Standardise : width = max, height = min
    width = max(w, h)
    height = min(w, h)
    return pd.Series([width, height])

df_core[["Res_Width", "Res_Height"]] = df_core["Screen Resolution"].apply(extract_resolution)

In [14]:
# Nombre total de pixels
df_core["Pixels_Total"] = df_core["Res_Width"] * df_core["Res_Height"]

# PPI (densité de pixels) si la taille en pouces est dispo
df_core["PPI"] = np.sqrt(
    df_core["Res_Width"]**2 + df_core["Res_Height"]**2
) / df_core["Screen_Size_Inch"]

In [15]:
# Refresh Rate → numérique
df_core["RefreshRate_Hz"] = (
    df_core["Refresh Rate"]
    .astype(str)
    .str.extract(r"(\d+)", expand=False)
)

df_core["RefreshRate_Hz"] = pd.to_numeric(df_core["RefreshRate_Hz"], errors="coerce")

In [16]:
# Brightness → numérique (Brightness_Nits) 
df_core["Brightness_Nits"] = (
    df_core["Brightness"]
    .astype(str)
    .str.extract(r"(\d+)", expand=False)
)

df_core["Brightness_Nits"] = pd.to_numeric(df_core["Brightness_Nits"], errors="coerce")

In [17]:
# Supprimer les anciennes colonnes texte d’écran
cols_drop_screen = [
    "Screen Size",
    "Screen Resolution",
    "Refresh Rate",
    "Brightness"
]

df_core.drop(columns=cols_drop_screen, inplace=True, errors="ignore")

In [18]:
# petit check 

df_core[[
    "Screen_Size_Inch", "Screen_Size_cm",
    "Res_Width", "Res_Height",
    "Pixels_Total", "PPI",
    "RefreshRate_Hz", "Brightness_Nits"
]].head()

,Screen_Size_Inch,Screen_Size_cm,Res_Width,Res_Height,Pixels_Total,PPI,RefreshRate_Hz,Brightness_Nits
0,17.3,43.942,2560.0,1440.0,3686400.0,169.780899,NaN,NaN
1,15.6,39.624,2560.0,1440.0,3686400.0,188.282664,NaN,NaN
2,16.1,40.894,1920.0,1080.0,2073600.0,136.826532,NaN,NaN
3,15.6,39.624,1920.0,1080.0,2073600.0,141.211998,NaN,250.0
4,14.0,35.560,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# petit check
import re

# 1) user rating -> numérique
df_core["user rating"] = pd.to_numeric(df_core["user rating"], errors="coerce")

# 2) Number of Cores -> numérique
df_core["Number of Cores"] = pd.to_numeric(df_core["Number of Cores"], errors="coerce")

# 3) Cache (en MB) -> Cache_MB
df_core["Cache_MB"] = (
    df_core["Cache"]
    .astype(str)
    .str.extract(r"(\d+)", expand=False)
)
df_core["Cache_MB"] = pd.to_numeric(df_core["Cache_MB"], errors="coerce")

# 4) RAM Frequency (MHz) -> RAM_Frequency_MHz
df_core["RAM_Frequency_MHz"] = (
    df_core["RAM Frequency"]
    .astype(str)
    .str.extract(r"(\d+)", expand=False)
)
df_core["RAM_Frequency_MHz"] = pd.to_numeric(df_core["RAM_Frequency_MHz"], errors="coerce")

# 5) Memory Slots -> Memory_Slots_Count
df_core["Memory_Slots_Count"] = (
    df_core["Memory Slots"]
    .astype(str)
    .str.extract(r"(\d+)", expand=False)
)
df_core["Memory_Slots_Count"] = pd.to_numeric(df_core["Memory_Slots_Count"], errors="coerce")

# 6) Battery Cell -> Battery_Cell_Count
df_core["Battery_Cell_Count"] = (
    df_core["Battery Cell"]
    .astype(str)
    .str.extract(r"(\d+)", expand=False)
)
df_core["Battery_Cell_Count"] = pd.to_numeric(df_core["Battery_Cell_Count"], errors="coerce")

# 7) TGP -> TGP_W (Watts)
df_core["TGP_W"] = (
    df_core["TGP"]
    .astype(str)
    .str.extract(r"([\d\.]+)", expand=False)
)
df_core["TGP_W"] = pd.to_numeric(df_core["TGP_W"], errors="coerce")

# 8) Clock Speed -> Clock_Base_GHz, Clock_Turbo_GHz
def extract_clock_speeds(text):
    if pd.isna(text):
        return pd.Series([np.nan, np.nan])
    s = str(text)
    # toutes les valeurs suivies de GHz/Ghz
    nums = re.findall(r"([\d\.]+)\s*G[Hh][Zz]", s)
    if not nums:
        return pd.Series([np.nan, np.nan])
    vals = [float(x.replace(",", ".")) for x in nums]
    base = vals[0]
    turbo = max(vals) if len(vals) > 1 else np.nan
    return pd.Series([base, turbo])

df_core[["Clock_Base_GHz", "Clock_Turbo_GHz"]] = df_core["Clock Speed"].apply(extract_clock_speeds)

# 9) Battery Backup -> Battery_Hours (approx, prend moyenne si "4-5 heures")
def extract_hours(text):
    if pd.isna(text):
        return np.nan
    s = str(text)
    # Ne récupère que des nombres du style 4, 4.5, 10.0, etc.
    nums = re.findall(r"\d+(?:\.\d+)?", s)
    if not nums:
        return np.nan
    vals = [float(x) for x in nums]
    # S'il y a "4-5 heures" -> moyenne (4.5)
    return sum(vals) / len(vals)

df_core["Battery_Hours"] = df_core["Battery Backup"].apply(extract_hours)
# 10) Supprimer les anciennes colonnes texte correspondantes
cols_drop_more = [
    "Cache",
    "RAM Frequency",
    "Memory Slots",
    "Battery Cell",
    "TGP",
    "Clock Speed",
    "Battery Backup"
]

df_core.drop(columns=cols_drop_more, inplace=True, errors="ignore")

# Petit check
df_core[[
    "user rating",
    "Number of Cores",
    "Cache_MB",
    "RAM_Frequency_MHz",
    "Memory_Slots_Count",
    "Battery_Cell_Count",
    "TGP_W",
    "Clock_Base_GHz", "Clock_Turbo_GHz",
    "Battery_Hours"
]].head()

,user rating,Number of Cores,Cache_MB,RAM_Frequency_MHz,Memory_Slots_Count,Battery_Cell_Count,TGP_W,Clock_Base_GHz,Clock_Turbo_GHz,Battery_Hours
0,5.0,14.0,24.0,NaN,NaN,4.0,NaN,2.5,5.0,NaN
1,NaN,14.0,24.0,NaN,NaN,4.0,NaN,2.5,5.0,NaN
2,NaN,8.0,16.0,NaN,NaN,4.0,NaN,3.2,4.4,NaN
3,NaN,8.0,16.0,3200.0,2.0,NaN,NaN,3.2,NaN,4.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0


In [20]:
df_core[[
    "user rating",
    "Number of Cores",
    "Cache_MB",
    "RAM_Frequency_MHz",
    "Memory_Slots_Count",
    "Battery_Cell_Count",
    "TGP_W",
    "Clock_Base_GHz", "Clock_Turbo_GHz",
    "Battery_Hours"
]].head(20)

,user rating,Number of Cores,Cache_MB,RAM_Frequency_MHz,Memory_Slots_Count,Battery_Cell_Count,TGP_W,Clock_Base_GHz,Clock_Turbo_GHz,Battery_Hours
0,5.0,14.0,24.0,NaN,NaN,4.0,NaN,2.5,5.0,NaN
1,NaN,14.0,24.0,NaN,NaN,4.0,NaN,2.5,5.0,NaN
2,NaN,8.0,16.0,NaN,NaN,4.0,NaN,3.2,4.4,NaN
3,NaN,8.0,16.0,3200.0,2.0,NaN,NaN,3.2,NaN,4.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0
5,NaN,4.0,8.0,3200.0,NaN,NaN,NaN,2.4,NaN,15.0
6,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0
7,NaN,8.0,8.0,3200.0,NaN,NaN,NaN,1.8,NaN,18.0
8,4.7,4.0,8.0,2933.0,4.0,NaN,NaN,2.5,NaN,5.0
9,4.0,4.0,4.0,2666.0,2.0,3.0,NaN,NaN,NaN,7.0


In [21]:
# ============================
# PHASE 4 - Variables d'aide
# ============================

import numpy as np
import re

# --- GPU : mémoire dédiée (texte) -> booléens utiles ---

# GPU dédié : on considère qu'il y a un GPU dédié si une capacité est renseignée
df_core["Has_Dedicated_GPU"] = df_core["Dedicated Graphic Memory Capacity"].notna()

# GPU puissant : RTX ou VRAM élevée (>= 6 ou 8 Go)
gpu_name = df_core["Graphic Processor"].astype(str)
gpu_vram_str = df_core["Dedicated Graphic Memory Capacity"].astype(str)

cond_rtx = gpu_name.str.contains("RTX", case=False, na=False)
cond_vram_high = gpu_vram_str.str.contains(r"6 GB|8 GB|16 GB", case=False, na=False)

df_core["Has_Powerful_GPU"] = cond_rtx | cond_vram_high

# GPU intégré : pas de mémoire dédiée OU mention "Integrated"
df_core["Has_Integrated_GPU"] = (
    (~df_core["Has_Dedicated_GPU"]) |
    gpu_name.str.contains("Integrated", case=False, na=False)
)

# --- CPU : indicateurs de puissance ---

cpu_name = df_core["Processor Name"].astype(str)
cpu_variant = df_core["Processor Variant"].astype(str)

# CPU très puissant (haut de gamme)
is_high_end_cpu = (
    cpu_name.str.contains(r"i9|Ryzen 9", case=False, regex=True) |
    (df_core["Number of Cores"] >= 8)
)

# CPU milieu/haut de gamme
is_upper_mid_cpu = (
    cpu_name.str.contains(r"i7|Ryzen 7", case=False, regex=True) |
    (df_core["Number of Cores"] >= 6)
)

# CPU d'entrée de gamme
is_low_cpu = cpu_name.str.contains(
    r"Celeron|Pentium|Athlon|i3|Ryzen 3", case=False, regex=True
)

# --- Suitable For normalisé en minuscule pour l'analyse texte ---

suitable_str = df_core["Suitable For"].astype(str).str.lower()

In [22]:
# ============================
# PHASE 4 - Primary_Usage
# ============================

# On initialise la colonne vide
df_core["Primary_Usage"] = np.nan

# 1) Graphisme : GPU puissant + écran QHD (largeur >= 2560)
cond_graphisme = df_core["Has_Powerful_GPU"] & (df_core["Res_Width"] >= 2560)
df_core.loc[cond_graphisme, "Primary_Usage"] = "Graphisme"

# 2) Gaming : GPU dédié + RAM >= 16 (sur les lignes encore non étiquetées)
cond_gaming = (
    df_core["Primary_Usage"].isna() &
    df_core["Has_Dedicated_GPU"] &
    (df_core["RAM_GB"] >= 16)
)
df_core.loc[cond_gaming, "Primary_Usage"] = "Gaming"

# 3) Développement : CPU puissant + RAM >= 16 (sur les lignes encore non étiquetées)
cond_dev = (
    df_core["Primary_Usage"].isna() &
    (is_high_end_cpu | is_upper_mid_cpu) &
    (df_core["RAM_GB"] >= 16)
)
df_core.loc[cond_dev, "Primary_Usage"] = "Développement"

# 4) Études : RAM <= 8 + GPU intégré (sur les lignes encore non étiquetées)
cond_etudes = (
    df_core["Primary_Usage"].isna() &
    (df_core["RAM_GB"] <= 8) &
    df_core["Has_Integrated_GPU"]
)
df_core.loc[cond_etudes, "Primary_Usage"] = "Études"

# 5) Utiliser 'Suitable For' comme secours pour les lignes restantes
#    - "gaming" -> Gaming
df_core.loc[
    df_core["Primary_Usage"].isna() & suitable_str.str.contains("gaming"),
    "Primary_Usage"
] = "Gaming"

#    - "processing & multitasking" ou "performance" -> Développement
df_core.loc[
    df_core["Primary_Usage"].isna() &
    (suitable_str.str.contains("processing & multitasking") |
     suitable_str.str.contains("performance")),
    "Primary_Usage"
] = "Développement"

#    - "everyday use", "entertainment", "travel & business" -> Études
df_core.loc[
    df_core["Primary_Usage"].isna() &
    (suitable_str.str.contains("everyday use") |
     suitable_str.str.contains("entertainment") |
     suitable_str.str.contains("travel & business")),
    "Primary_Usage"
] = "Études"

# 6) Dernier recours : tout ce qui reste non défini -> Études
df_core["Primary_Usage"].fillna("Études", inplace=True)

# Vérification rapide
df_core[["name", "RAM_GB", "Graphic Processor", "Res_Width", "Primary_Usage"]].head(10)

C:\Users\Monsieur\AppData\Local\Temp\ipykernel_18216\716244435.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Graphisme' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_core.loc[cond_graphisme, "Primary_Usage"] = "Graphisme"
C:\Users\Monsieur\AppData\Local\Temp\ipykernel_18216\716244435.py:61: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_core["Primary_Usage"].fillna("Études", inplace=True)


,name,RAM_GB,Graphic Processor,Res_Width,Primary_Usage
0,ASUS ROG Strix SCAR 17 Core i9 12th Gen - (32 ...,32,NVIDIA GeForce RTX 3070 Ti,2560.0,Graphisme
1,ASUS ROG Strix SCAR 15 Core i9 12th Gen - (32 ...,32,NVIDIA GeForce RTX 3070 Ti,2560.0,Graphisme
2,HP Victus Ryzen 7 Octa Core 5800H - (16 GB/512...,16,NVIDIA GeForce RTX 3050,1920.0,Gaming
3,Lenovo IdeaPad Gaming 3i Ryzen 7 Octa Core R7-...,16,NVIDIA GeForce RTX 3050,1920.0,Gaming
4,Lenovo Yoga Slim 7 Core i5 11th Gen - (16 GB/5...,16,Intel Integrated Intel,NaN,Développement
5,Lenovo Yoga Slim 7 Core i7 11th Gen - (16 GB/5...,16,Intel Integrated Iris Xe Graphics,1920.0,Développement
6,Lenovo Lenovo V15 Celeron Dual Core - (4 GB/25...,4,Intel Integrated Intel HD,NaN,Études
7,Lenovo Yoga 6 Ryzen 7 Octa Core R7-5700U 5th G...,16,AMD Radeon Integrated AMD Radeon Graphics,1920.0,Développement
8,ASUS TUF Gaming F15 Core i5 10th Gen - (8 GB/1...,8,NVIDIA GeForce GTX 1650,1920.0,Gaming
9,DELL Inspiron Pentium Silver - (8 GB/256 GB SS...,8,Intel Integrated UHD Graphics,1920.0,Études


In [23]:
# ============================
# PHASE 4 - Performance_Level
# ============================

# --- GPU_VRAM_GB (numérique) ---
df_core["GPU_VRAM_GB"] = (
    df_core["Dedicated Graphic Memory Capacity"]
    .astype(str)
    .str.extract(r"(\d+)", expand=False)
)
df_core["GPU_VRAM_GB"] = pd.to_numeric(df_core["GPU_VRAM_GB"], errors="coerce")

# --- Scores CPU / GPU / RAM en 0..3 ---

# CPU_Score
df_core["CPU_Score"] = 2  # valeur par défaut = moyen
df_core.loc[is_low_cpu, "CPU_Score"] = 1
df_core.loc[is_upper_mid_cpu, "CPU_Score"] = 2
df_core.loc[is_high_end_cpu, "CPU_Score"] = 3

# GPU_Score
df_core["GPU_Score"] = 0
df_core.loc[df_core["GPU_VRAM_GB"].between(1, 3, inclusive="both"), "GPU_Score"] = 1
df_core.loc[df_core["GPU_VRAM_GB"].between(4, 5, inclusive="both"), "GPU_Score"] = 2
df_core.loc[df_core["GPU_VRAM_GB"] >= 6, "GPU_Score"] = 3

# RAM_Score
df_core["RAM_Score"] = 0
df_core.loc[df_core["RAM_GB"] <= 8, "RAM_Score"] = 1
df_core.loc[df_core["RAM_GB"].between(9, 16, inclusive="both"), "RAM_Score"] = 2
df_core.loc[df_core["RAM_GB"] >= 32, "RAM_Score"] = 3

# Score global pondéré (max théorique = 3)
df_core["Perf_Score"] = (
    0.4 * df_core["CPU_Score"].fillna(0) +
    0.4 * df_core["GPU_Score"].fillna(0) +
    0.2 * df_core["RAM_Score"].fillna(0)
)

# Mapping Perf_Score -> Performance_Level
def perf_level(score):
    if pd.isna(score):
        return np.nan
    if score < 1.5:
        return "Low"
    elif score < 2.2:
        return "Medium"
    elif score < 2.7:
        return "High"
    else:
        return "Extreme"

df_core["Performance_Level"] = df_core["Perf_Score"].apply(perf_level)

# Vérification
df_core[["name", "CPU_Score", "GPU_Score", "RAM_Score", "Perf_Score", "Performance_Level"]].head(10)

,name,CPU_Score,GPU_Score,RAM_Score,Perf_Score,Performance_Level
0,ASUS ROG Strix SCAR 17 Core i9 12th Gen - (32 ...,3,3,3,3.0,Extreme
1,ASUS ROG Strix SCAR 15 Core i9 12th Gen - (32 ...,3,3,3,3.0,Extreme
2,HP Victus Ryzen 7 Octa Core 5800H - (16 GB/512...,3,2,2,2.4,High
3,Lenovo IdeaPad Gaming 3i Ryzen 7 Octa Core R7-...,3,2,2,2.4,High
4,Lenovo Yoga Slim 7 Core i5 11th Gen - (16 GB/5...,2,0,2,1.2,Low
5,Lenovo Yoga Slim 7 Core i7 11th Gen - (16 GB/5...,2,0,2,1.2,Low
6,Lenovo Lenovo V15 Celeron Dual Core - (4 GB/25...,1,0,1,0.6,Low
7,Lenovo Yoga 6 Ryzen 7 Octa Core R7-5700U 5th G...,3,0,2,1.6,Medium
8,ASUS TUF Gaming F15 Core i5 10th Gen - (8 GB/1...,2,2,1,1.8,Medium
9,DELL Inspiron Pentium Silver - (8 GB/256 GB SS...,1,0,1,0.6,Low


In [24]:
# ============================
# PHASE 4 - Estimated_Consumption & Eco_Class
# ============================

# Valeur par défaut : Medium
df_core["Estimated_Consumption"] = "Medium"

# High : GPU puissant + grand écran (>= 15.6")
cond_high_cons = df_core["Has_Powerful_GPU"] & (df_core["Screen_Size_Inch"] >= 15.6)
df_core.loc[cond_high_cons, "Estimated_Consumption"] = "High"

# Low : laptop léger (<= 1.5 kg) + pas de GPU dédié
cond_low_cons = (df_core["Weight_KG"] <= 1.5) & (~df_core["Has_Dedicated_GPU"])
df_core.loc[cond_low_cons, "Estimated_Consumption"] = "Low"

# Eco_Class à partir de Estimated_Consumption
def map_eco(cons):
    if cons == "Low":
        return "A"
    elif cons == "Medium":
        return "B"
    elif cons == "High":
        return "C"
    else:
        return np.nan

df_core["Eco_Class"] = df_core["Estimated_Consumption"].apply(map_eco)

# Vérification
df_core[["name", "Screen_Size_Inch", "Weight_KG", "Has_Powerful_GPU",
         "Estimated_Consumption", "Eco_Class"]].head(10)

,name,Screen_Size_Inch,Weight_KG,Has_Powerful_GPU,Estimated_Consumption,Eco_Class
0,ASUS ROG Strix SCAR 17 Core i9 12th Gen - (32 ...,17.3,2.90,True,High,C
1,ASUS ROG Strix SCAR 15 Core i9 12th Gen - (32 ...,15.6,2.30,True,High,C
2,HP Victus Ryzen 7 Octa Core 5800H - (16 GB/512...,16.1,2.48,True,High,C
3,Lenovo IdeaPad Gaming 3i Ryzen 7 Octa Core R7-...,15.6,2.25,True,High,C
4,Lenovo Yoga Slim 7 Core i5 11th Gen - (16 GB/5...,14.0,NaN,False,Medium,B
5,Lenovo Yoga Slim 7 Core i7 11th Gen - (16 GB/5...,14.0,1.36,False,Low,A
6,Lenovo Lenovo V15 Celeron Dual Core - (4 GB/25...,15.6,NaN,False,Medium,B
7,Lenovo Yoga 6 Ryzen 7 Octa Core R7-5700U 5th G...,13.3,1.31,False,Low,A
8,ASUS TUF Gaming F15 Core i5 10th Gen - (8 GB/1...,15.6,2.30,False,Medium,B
9,DELL Inspiron Pentium Silver - (8 GB/256 GB SS...,15.6,1.61,False,Medium,B


In [29]:
# Nettoyage du prix indien
df_core["Price_INR"] = (
    df_core["Price"]
    .astype(str)
    .str.replace("₹", "", regex=False)
    .str.replace("?", "", regex=False)
    .str.replace(",", "", regex=False)
    .astype(float)
)

# Conversion en dirhams

INR_TO_MAD = 0.12  # taux fixe pédagogique

df_core["Price_MAD"] = (df_core["Price_INR"] * INR_TO_MAD).round(0)


In [31]:
# catégorisation de prix basée sur la distribution réelle des données, à partir de Price_MAD.
df_core["Price_MAD"].describe()


count      984.000000
mean      8382.132114
std       4643.279420
min       1919.000000
25%       5217.250000
50%       7199.000000
75%       9895.500000
max      36239.000000
Name: Price_MAD, dtype: float64

In [34]:
#SPLIT PAR QUANTILES (LE PLUS RAPIDE & PROPRE)
df_core["Price_Quantile"] = pd.qcut(
    df_core["Price_MAD"],
    q=4,
    labels=["Q1", "Q2", "Q3", "Q4"]
)


In [35]:
price_tier_map = {
    "Q1": "Entry",
    "Q2": "Mid",
    "Q3": "High",
    "Q4": "Premium"
}

df_core["Price_Tier"] = df_core["Price_Quantile"].map(price_tier_map)


In [36]:
df_core["Price_Tier"].value_counts()


Price_Tier
Mid        250
Entry      246
Premium    246
High       242
Name: count, dtype: int64

La catégorisation des prix a été réalisée à l’aide d’un découpage par quantiles, garantissant une répartition équilibrée des ordinateurs selon leur distribution réelle.
Cette approche permet d’éviter toute définition arbitraire des seuils de prix et assure une meilleure robustesse du moteur de recommandation.

In [38]:
output_path = r"C:\Users\Monsieur\OneDrive\Desktop\UNIVERSITE S7\NoSql & BIG DATA\LapMatch\data\laptops_core_final1.csv"

df_core.to_csv(output_path, index=False, encoding="utf-8")

# Phase 5 – Structuration du dataset pour la recommandation

Dans cette phase, le dataset final est structuré de manière logique afin de distinguer les variables de filtrage (budget, usage, préférences utilisateur), les variables de scoring (performance CPU/GPU, RAM, stockage) et les variables informatives (classe écologique, consommation estimée). Cette organisation facilite l’implémentation du moteur de recommandation dans le backend FastAPI et prépare la future transformation des données vers un modèle orienté graphe avec Neo4j.

In [39]:
# ============================
# PHASE 5 - CSV PIVOT FINAL
# ============================

df_phase5 = pd.DataFrame()

# --- Identité Laptop ---
df_phase5["laptop_id"] = df_core.index.astype(str)
df_phase5["name"] = df_core["name"]
df_phase5["external_link"] = df_core["link"]

# --- Prix & rating ---
df_phase5["price"] = df_core["Price_MAD"]
df_phase5["price_tier"] = df_core["Price_Tier"]
df_phase5["rating"] = df_core["user rating"]

# --- Specs clés ---
df_phase5["ram_gb"] = df_core["RAM_GB"]
df_phase5["storage_gb"] = df_core["Storage_GB"]
df_phase5["weight"] = df_core["Weight_KG"]
df_phase5["screen_size"] = df_core["Screen_Size_Inch"]

# --- CPU / GPU (lisibles + scores) ---
df_phase5["cpu"] = (
    df_core["Processor Brand"].fillna("") + " " +
    df_core["Processor Name"].fillna("") + " " +
    df_core["Processor Generation"].fillna("")
).str.strip()

df_phase5["gpu"] = df_core["Graphic Processor"].fillna("Integrated")
df_phase5["gpu_vram_gb"] = df_core["GPU_VRAM_GB"]

df_phase5["cpu_score"] = df_core["CPU_Score"]
df_phase5["gpu_score"] = df_core["GPU_Score"]
df_phase5["ram_score"] = df_core["RAM_Score"]
df_phase5["perf_score"] = df_core["Perf_Score"]
df_phase5["performance_level"] = df_core["Performance_Level"]

# --- Usage ---
df_phase5["primary_usage"] = df_core["Primary_Usage"]

# --- Écologie ---
df_phase5["eco_class"] = df_core["Eco_Class"]
df_phase5["estimated_consumption"] = df_core["Estimated_Consumption"]

# --- Prix source (traçabilité) ---
df_phase5["price_inr"] = df_core["Price_INR"]


In [40]:
df_phase5.info()
df_phase5.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   laptop_id              984 non-null    object  
 1   name                   984 non-null    object  
 2   external_link          984 non-null    object  
 3   price                  984 non-null    float64 
 4   price_tier             984 non-null    category
 5   rating                 690 non-null    float64 
 6   ram_gb                 984 non-null    int64   
 7   storage_gb             961 non-null    float64 
 8   weight                 872 non-null    float64 
 9   screen_size            984 non-null    float64 
 10  cpu                    984 non-null    object  
 11  gpu                    984 non-null    object  
 12  gpu_vram_gb            310 non-null    float64 
 13  cpu_score              984 non-null    int64   
 14  gpu_score              984 non-null    int

,laptop_id,name,external_link,price,price_tier,rating,ram_gb,storage_gb,weight,screen_size,cpu,gpu,gpu_vram_gb,cpu_score,gpu_score,ram_score,perf_score,performance_level,primary_usage,eco_class,estimated_consumption,price_inr
0,0,ASUS ROG Strix SCAR 17 Core i9 12th Gen - (32 ...,https://www.flipkart.com/asus-rog-strix-scar-1...,28199.0,Premium,5.0,32,1024.0,2.90,17.3,Intel Core i9 12th Gen,NVIDIA GeForce RTX 3070 Ti,8.0,3,3,3,3.0,Extreme,Graphisme,C,High,234990.0
1,1,ASUS ROG Strix SCAR 15 Core i9 12th Gen - (32 ...,https://www.flipkart.com/asus-rog-strix-scar-1...,27599.0,Premium,NaN,32,1024.0,2.30,15.6,Intel Core i9 12th Gen,NVIDIA GeForce RTX 3070 Ti,8.0,3,3,3,3.0,Extreme,Graphisme,C,High,229990.0
2,2,HP Victus Ryzen 7 Octa Core 5800H - (16 GB/512...,https://www.flipkart.com/hp-victus-ryzen-7-oct...,12491.0,Premium,NaN,16,512.0,2.48,16.1,AMD Ryzen 7 Octa Core,NVIDIA GeForce RTX 3050,4.0,3,2,2,2.4,High,Gaming,C,High,104091.0
3,3,Lenovo IdeaPad Gaming 3i Ryzen 7 Octa Core R7-...,https://www.flipkart.com/lenovo-ideapad-gaming...,10526.0,Premium,NaN,16,512.0,2.25,15.6,AMD Ryzen 7 Octa Core 5th Gen,NVIDIA GeForce RTX 3050,4.0,3,2,2,2.4,High,Gaming,C,High,87717.0
4,4,Lenovo Yoga Slim 7 Core i5 11th Gen - (16 GB/5...,https://www.flipkart.com/lenovo-yoga-slim-7-co...,9119.0,High,NaN,16,512.0,NaN,14.0,Intel Core i5 11th Gen,Intel Integrated Intel,NaN,2,0,2,1.2,Low,Développement,B,Medium,75990.0


In [41]:
output_path = "./data/lapmatch_phase5_final.csv"

df_phase5.to_csv(
    output_path,
    index=False,
    encoding="utf-8"
)

print(f"CSV Phase 5 sauvegardé : {output_path}")


CSV Phase 5 sauvegardé : ./data/lapmatch_phase5_final.csv


La Phase 5 produit un CSV pivot aligné avec le modèle orienté graphe.
Chaque colonne correspond directement à une entité ou une relation Neo4j, ce qui permet un split simple et sans retraitement vers les nœuds Laptop, Brand, Usage, Component et Eco_Class.